# Get Vacancy by Distance

In [1]:
# get clean vacant
import csv
import pandas as pd
vacant_Baltimore = pd.read_csv('CleanDataBase/vacantBaltimore_171117_8000.csv',sep='\t') # create header later
vacant_Baltimore = vacant_Baltimore.drop("Unnamed: 0", 1)

# clean street name
vacant_Baltimore.loc[:,"street"] = vacant_Baltimore.loc[:,"street"].str[1:-1]
print "number of Vacancy in db25:", len(vacant_Baltimore)

number of Vacancy in db25: 8602


In [2]:
# get osmnx GEOdataframe
import osmnx as ox
address = '1516 Kenhill Ave, Baltimore, MD'
radius = 700
gdf = ox.buildings_from_address(address, distance=radius)

print "Address:", address
print "radius", radius
print "number of buildings from osmnx:", len(gdf)

Address: 1516 Kenhill Ave, Baltimore, MD
radius 700
number of buildings from osmnx: 3716


In [3]:
## check iff street is not null and building type is yes
result = pd.concat([gdf["addr:street"].isnull() == False, gdf["building"] == "yes" ], axis=1)
result["sum"] = result["addr:street"] & result["building"]
street_gdf = gdf[result["sum"]]

print "number of buildings with street name is not null:", len(street_gdf)

number of buildings with street name is not null: 3359


In [4]:
def vacant_generator(i,j):
    
    while i < len(street_gdf.index):
        while j < len(vacant_Baltimore.index):
            if (street_gdf['addr:street'][street_gdf.index[i]] == vacant_Baltimore['street'][vacant_Baltimore.index[j]] and 
                street_gdf['addr:housenumber'][street_gdf.index[i]] == vacant_Baltimore['housenumber'][vacant_Baltimore.index[j]]):
                
                yield street_gdf.index[i],vacant_Baltimore.index[j], street_gdf['addr:street'][street_gdf.index[i]], street_gdf['addr:housenumber'][street_gdf.index[i]]
            yield street_gdf.index[i],vacant_Baltimore.index[j],None,None
            j += 1
        
        if i%1000==0:
            print i
            
        i += 1
        j = 0
        
                #m.gdf.loc[i,'housetype'] = 2
            #elif m.gdf['housetype'][i] == 2:
                #m.gdf.loc[i,'housetype'] = 1

In [5]:
vacantOSMNX = []
vacantdb25 = []

for i,j,street,housenumber in vacant_generator(0,0):
    if street != None:
        print street, housenumber, i, j
        vacantOSMNX.append(i)
        vacantdb25.append(j)

0
Edison Highway 1218 336237580 7041
North Ellwood Avenue 1330 336241834 4551
East Federal Street 3024 336242110 3063
North Decker Avenue 1239 336243146 663
North Decker Avenue 1210 336243636 2821
North Potomac Street 1241 336244932 2425
North Potomac Street 1401 336245111 4190
East Federal Street 3011 336245341 3706
North Potomac Street 1224 336245812 1084
North Potomac Street 1234 336245820 6989
North Potomac Street 1220 336245822 6646
North Potomac Street 1428 336246839 4175
North Curley Street 1221 336247215 2741
North Linwood Avenue 1239 336249037 2893
North Linwood Avenue 1227 336249040 8317
North Linwood Avenue 1229 336249045 2437
East Biddle Street 2833 336251498 8534
East Chase Street 2828 336251630 3971
North Kenwood Avenue 1015 336252586 4891
East Federal Street 2818 336253554 4038
North Kenwood Avenue 1507 336253917 5884
East Preston Street 2730 336254641 7271
1000
Mura Street 2701 336255557 1740
Mura Street 2705 336255563 8341
Mura Street 2731 336255564 6265
East Biddle St

East Preston Street 2322 336268414 5601
East Preston Street 2316 336268418 996
North Gay Street 1831 336268544 6427
North Montford Avenue 1804 336268547 2953
North Montford Avenue 1810 336268548 2951
North Montford Avenue 1806 336268550 2684
North Montford Avenue 1808 336268551 4475
East Oliver Street 2330 336268660 5680
North Montford Avenue 1720 336268841 1541
North Montford Avenue 1722 336268843 758
North Montford Avenue 1704 336268848 767
North Montford Avenue 1712 336268850 5738
North Bradford Street 1625 336269215 2946
North Bradford Street 1631 336269219 793
North Bradford Street 1617 336269221 2730
North Bradford Street 1619 336269227 2733
East Chase Street 2300 336269603 8364
East Lafayette Avenue 2317 336269620 2697
North Bradford Street 1511 336269664 4482
East Hoffman Street 2309 336269746 1171
East Oliver Street 2331 336269751 3848
East Oliver Street 2335 336269756 2740
East Oliver Street 2309 336269783 3842
North Bradford Street 1719 336269846 5356
3000
North Bradford Str

In [6]:
len(vacantdb25)

273

In [7]:
# write data into csv
with open('vacantOSMNX_1516KenhillAve700_171117', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(vacantOSMNX)
    
"""
# Read data from csv
with open('vacantOSMNX_Baltimore_171117', 'rb') as f:
    reader = csv.reader(f)
    your_list = map(int,list(reader)[0])
"""

"\n# Read data from csv\nwith open('vacantOSMNX_Baltimore_171117', 'rb') as f:\n    reader = csv.reader(f)\n    your_list = map(int,list(reader)[0])\n"